In [113]:
# import bernstein.py
import numpy as np
import math
import scipy.special as sp
import sympy as sym
import math
import p3basis as b

In [114]:
# BINOMIAL COEFFICIENT SHORTCUT
def C(n,k):
    return sp.binom(n, k)
# C(0,0) # 1
# C(0,1) # 0
# C(1,0) # 1
# C(1,1) # 1
# C(2,0) # 1
C(2,1) # 2

2.0

In [115]:
# EXISTING BERNSTEIN EVAL FUNCTION
def eval_B(xmin, xmax, deg, B_idx, x):
    xi = b.map_x_to_xi(xmin, xmax, x, basis_lower=0, basis_upper=1)
    i = B_idx
    n = deg
    c_term = C(n,i)
    print("c_term:", c_term)
    nix_term = (1 - x)**(n - i) * x**i
    print("nix_term:", nix_term)
    return c_term * nix_term

# eval_B(xmin=0, xmax=1, deg=2, B_idx=0, x=0.75)
#// idx 0 @ 0 = 1
#// idx 1 @ 0 = 0
#// idx 2 @ 0 = 0
#// idx 0 @ 1 = 0
#// idx 1 @ 1 = 0
#// idx 2 @ 1 = 1
#// idx 0 @ 0.5 = 0.25
#// idx 1 @ 0.5 = 0.5
#// idx 2 @ 0.5 = 0.25
#// idx 0 @ 0.5 = 0.25
#// idx 1 @ 0.5 = 0.5
# // idx 2 @ 0.5 = 0.25

# deg=2 at arbitrary points
# per https://www.geogebra.org/m/x8VjQkjm?doneurl=%2Fmat4u
# idx 0 @ 0.75 = 0.0625
# idx 1 @ 0.75 = 0.375
# idx 2 @ 0.75 = 0.5625
# OK, ALL ABOVE CONFIRMED CORRECT

eval_B(xmin=0, xmax=1, deg=1, B_idx=0, x=0)
# eval_B(xmin=0, xmax=1, deg=2, B_idx=2, x=0.75)



c_term: 1.0
nix_term: 1


1.0

In [116]:
# FORMULA FOR DERIVATIVE OF C TERM
    # https://math.stackexchange.com/questions/1941994/derivative-of-the-binomial-binom-x-n-with-respect-to-x
    # deriv C(x,n) = sum k=1 to n, (((-1)^{k-1})/k) * C(x, n-k)
    # TRANSLATED TO n,i
    # sum k=1 to i, (((-1)^{k-1})/k) * C(x, i-k)

# NOW TO CODE THE DERIVATIVE OF THE C-TERM
def C_prime(p, A):
    sum = 0
    for j in range(1, A+1):
        sum += ((-1)**(j-1) / j) * C(p, A-j)
    return sum


# C(3, 0), C_prime(3, 0)
# TODO: COMPARE TO FORMULA GIVEN IN CLASS

In [117]:
# DERIVATIVE OF THE NON=BINOMIAL TERM OF THE BERNSTEIN EQN
t,n,i = sym.symbols('t,n i')
# def B_eqn_base(t,n,i):
#     return (1 - t)**(n - i) * t**i
# def B_eqn_base_prime(t,n,i):
#     return sym.diff(B_eqn_base(t,n,i),t)
# # print(B_eqn_base_prime(t,n,i)) # this will print the expression
# B_eqn_base_prime = sym.lambdify((t,n,i),B_eqn_base_prime(t,n,i),"numpy")

def B_eqn_nix(t,n,i):
    return (1 - t)**(n - i)
def B_eqn_x(t,n,i):
    return t**i
def B_eqn_nix_prime(t,n,i):
    return sym.diff(B_eqn_nix(t,n,i),t)
def B_eqn_x_prime(t,n,i):
    return sym.diff(B_eqn_x(t,n,i),t)
# print(B_eqn_base_prime(t,n,i)) # this will print the expression
B_nix_prime = sym.lambdify((t,n,i),B_eqn_nix_prime(t,n,i),"numpy")
B_x_prime = sym.lambdify((t,n,i),B_eqn_x_prime(t,n,i),"numpy")

In [119]:

# ASSUMING THAT'S CORRECT, WE CAN NOW DO THE PRODUCT RULE ON THE BERNSTEIN FORMULA TO GET ITS DERIVATIVE
# d/dx[f(x)g(g)] = f(x)g'(x) + f'(x)g(x)
# bernstein formula is: c_term * (1 - x)**(n - i) * x**i, or 
# call this c_term * nix_term * x_term
# call product of nix and x terms nixx_term
# so product rule says:
    # c_term * nixx_term_prime + c_term_prime * nixx_term
    # where nix_term_prime is:
        # nix_term_prime = (nix_term * x_term_prime + nix_term_prime * x_term)
    # giving final equation of: 
        # c_term * (nix_term * x_term_prime + nix_term_prime * x_term) + c_term_prime * nixx_term

def eval_B_prime(xmin, xmax, deg, B_idx, x):
    # print("running eval_B_prime")
    xi =b.map_x_to_xi(xmin, xmax, x, basis_lower=0, basis_upper=1)
    print("xi=", xi)
    i = B_idx
    n = deg
    c_term = C(n,i)
    nix_term = B_eqn_nix(t=x, n=n, i=i)
    x_term = B_eqn_x(t=x, n=n, i=i)
    nixx_term = nix_term * x_term
    c_term_prime = C_prime(n,i)
    nix_term_prime = B_nix_prime(x, n, i)
    x_term_prime = B_x_prime(x, n, i)
    # print(nix_term_prime(t,n,i))
    # c_term * (nix_term * x_term_prime + nix_term_prime * x_term) + c_term_prime * nixx_term
    return (c_term * (nix_term * x_term_prime + nix_term_prime * x_term)) + c_term_prime * nixx_term

# TESTING: SINCE B00=1, DERIV SHOULD BE 0 FOR ANY X
B00 = eval_B(xmin=0, xmax=1, deg=0, B_idx=0, x=0.33)
B00
# B00_prime = eval_B_prime(xmin=0, xmax=1, deg=0, B_idx=0, x=0.1)
# B00_prime
# testing: since B01=t, deriv should be 1 for any x
# B01 = eval_B(xmin=0, xmax=1, deg=0, B_idx=1, x=0)
# print("B01, x=0.8",B01)
# B01_prime = eval_B_prime(xmin=0, xmax=1, deg=0, B_idx=1, x=0.8)
# B01_prime

# testing: since B22=t^2 and B22=t^2-2t+1, their derivs should be 2x and 2x-2, respectively.
# B22_prime_025 = eval_B_prime(xmin=0, xmax=1, deg=2, B_idx=2, x=0.25) 
# B22_prime_025 
# B22 = t^2, so B22_prime = 2t
# 2 * 0.25 = 0.5


# B2005 = eval_B_prime(xmin=0, xmax=1, deg=2, B_idx=0, x=0.75) # 2 * 0.75 - 2 = -0.25
# print(B22025, B2005) 


c_term: 1.0
nix_term: 1.0


1.0